Intalacoes necessarias

In [95]:
pip install ucimlrepo

In [96]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [97]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [98]:
pip install sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-

Importando as bibliotecas

In [130]:
from ucimlrepo import fetch_ucirepo 
from IPython.display import display
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import pandas as pd
import seaborn as sns

Coletando os datasets

In [100]:
pd.set_option('display.max_columns', None)
dataBase = pd.read_csv(r"https://archive.ics.uci.edu/static/public/887/data.csv")

Exibindo os datasets puro

In [101]:
# fetch dataset 
national_health_and_nutrition_health_survey_2013_2014_nhanes_age_prediction_subset = fetch_ucirepo(id=887) 
  
# data (as pandas dataframes) 
X = national_health_and_nutrition_health_survey_2013_2014_nhanes_age_prediction_subset.data.features 
y = national_health_and_nutrition_health_survey_2013_2014_nhanes_age_prediction_subset.data.targets 
  
# metadata 
print(national_health_and_nutrition_health_survey_2013_2014_nhanes_age_prediction_subset.metadata) 
  
# variable information 
print(national_health_and_nutrition_health_survey_2013_2014_nhanes_age_prediction_subset.variables) 

{'uci_id': 887, 'name': 'National Health and Nutrition Health Survey 2013-2014 (NHANES) Age Prediction Subset', 'repository_url': 'https://archive.ics.uci.edu/dataset/887/national+health+and+nutrition+health+survey+2013-2014+(nhanes)+age+prediction+subset', 'data_url': 'https://archive.ics.uci.edu/static/public/887/data.csv', 'abstract': "The National Health and Nutrition Examination Survey (NHANES), administered by the Centers for Disease Control and Prevention (CDC), collects extensive health and nutritional information from a diverse U.S. population. Though expansive, the dataset is often too broad for specific analytical purposes. In this sub-dataset, we narrow our focus to predicting respondents' age by extracting a subset of features from the larger NHANES dataset. These selected features include physiological measurements, lifestyle choices, and biochemical markers, which were hypothesized to have strong correlations with age.", 'area': 'Health and Medicine', 'tasks': ['Classifi

In [102]:
display(dataBase.head(10))

,SEQN,age_group,RIDAGEYR,RIAGENDR,PAQ605,BMXBMI,LBXGLU,DIQ010,LBXGLT,LBXIN
0,73564.0,Adult,61.0,2.0,2.0,35.7,110.0,2.0,150.0,14.91
1,73568.0,Adult,26.0,2.0,2.0,20.3,89.0,2.0,80.0,3.85
2,73576.0,Adult,16.0,1.0,2.0,23.2,89.0,2.0,68.0,6.14
3,73577.0,Adult,32.0,1.0,2.0,28.9,104.0,2.0,84.0,16.15
4,73580.0,Adult,38.0,2.0,1.0,35.9,103.0,2.0,81.0,10.92
5,73581.0,Adult,50.0,1.0,2.0,23.6,110.0,2.0,100.0,6.08
6,73587.0,Adult,14.0,1.0,2.0,38.7,94.0,2.0,202.0,21.11
7,73596.0,Adult,57.0,2.0,2.0,38.3,107.0,2.0,164.0,20.93
8,73607.0,Senior,75.0,1.0,2.0,38.9,89.0,2.0,113.0,17.47
9,73610.0,Adult,43.0,1.0,1.0,28.9,90.0,2.0,95.0,3.24


Tratamento da base

In [103]:
dataBase = dataBase.rename(columns={
    'SEQN': 'sk',
    'age_group': 'faixaEtaria',
    'RIDAGEYR': 'idade',
    'RIAGENDR': 'sexo',
    'PAQ605': 'exerciciosSemanais',
    'BMXBMI': 'imc',
    'LBXGLU': 'nivelGlicose',
    'DIQ010': 'diabetico',
    'LBXGLT': 'LBXGLT',
    'LBXIN': 'nivelInsulinaSangue'
})

In [106]:
dataBase["sexo"] = dataBase["sexo"].replace({1: 'MALE', 2: 'FEMALE'})
dataBase["exerciciosSemanais"] = dataBase["exerciciosSemanais"].replace({1: 'SIM', 2: 'NAO'})
dataBase["diabetico"] = dataBase["diabetico"].replace({1: 'SIM', 2: 'NAO'})

In [110]:
display(dataBase.head(10))

,sk,faixaEtaria,idade,sexo,exerciciosSemanais,imc,nivelGlicose,diabetico,LBXGLT,nivelInsulinaSangue
0,73564.0,Adult,61.0,FEMALE,NAO,35.7,110.0,NAO,150.0,14.91
1,73568.0,Adult,26.0,FEMALE,NAO,20.3,89.0,NAO,80.0,3.85
2,73576.0,Adult,16.0,MALE,NAO,23.2,89.0,NAO,68.0,6.14
3,73577.0,Adult,32.0,MALE,NAO,28.9,104.0,NAO,84.0,16.15
4,73580.0,Adult,38.0,FEMALE,SIM,35.9,103.0,NAO,81.0,10.92
5,73581.0,Adult,50.0,MALE,NAO,23.6,110.0,NAO,100.0,6.08
6,73587.0,Adult,14.0,MALE,NAO,38.7,94.0,NAO,202.0,21.11
7,73596.0,Adult,57.0,FEMALE,NAO,38.3,107.0,NAO,164.0,20.93
8,73607.0,Senior,75.0,MALE,NAO,38.9,89.0,NAO,113.0,17.47
9,73610.0,Adult,43.0,MALE,SIM,28.9,90.0,NAO,95.0,3.24


In [113]:
display(dataBase.describe())

,sk,idade,imc,nivelGlicose,LBXGLT,nivelInsulinaSangue
count,2278.000000,2278.000000,2278.000000,2278.000000,2278.000000,2278.000000
mean,78691.853819,41.795874,27.955180,99.553117,114.978929,11.834794
std,2921.365151,20.156111,7.248962,17.889834,47.061239,9.718812
min,73564.000000,12.000000,14.500000,63.000000,40.000000,0.140000
25%,76171.750000,24.000000,22.800000,91.000000,87.000000,5.860000
50%,78749.000000,41.000000,26.800000,97.000000,105.000000,9.040000
75%,81213.500000,58.000000,31.200000,104.000000,130.000000,14.440000
max,83727.000000,80.000000,70.100000,405.000000,604.000000,102.290000


Treinamento de teste

In [114]:
x = dataBase['idade']
y = dataBase[['imc', 'nivelGlicose']]

In [115]:
if x.isnull().any() or y.isnull().any().any():  # Verifica se há valores ausentes em y
    print("Valores ausentes encontrados. Removendo linhas com valores ausentes.")
    dataBase = dataBase.dropna(subset=['idade', 'imc', 'nivelGlicose'])  # Remover linhas com valores ausentes
    x = dataBase['idade']
    y = dataBase[['imc', 'nivelGlicose']]

In [116]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, 
    test_size=0.3, 
    random_state=123
)

In [117]:
print("Dimensões do conjunto de treino:")
print(f"x_train: {x_train.shape}, y_train: {y_train.shape}")
print("Dimensões do conjunto de teste:")
print(f"x_test: {x_test.shape}, y_test: {y_test.shape}")

Dimensões do conjunto de treino:
x_train: (1594,), y_train: (1594, 2)
Dimensões do conjunto de teste:
x_test: (684,), y_test: (684, 2)


In [131]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, 
    test_size=0.3, 
    random_state=123
)

# Criacao da saida multi regressao
regressor = MultiOutputRegressor(LinearRegression())

# Ajuste do modelo
regressor.fit(x_train.values.reshape(-1, 1), y_train)

# Previsões
y_pred = regressor.predict(x_test.values.reshape(-1, 1))

# Avaliacao do desempenho para cada saída
for i, coluna in enumerate(y_train.columns):
    mse = mean_squared_error(y_test[coluna], y_pred[:, i])
    print(f"Erro quadrático médio para {coluna}: {mse}")

Erro quadrático médio para imc: 50.378884502155785
Erro quadrático médio para nivelGlicose: 402.8259095253492
